In [ ]:
# import the necessary packages
# from pyimagesearch.transform import four_point_transform
from imutils.perspective import four_point_transform
from skimage.filters import threshold_local
import numpy as np
import argparse
import cv2
import imutils

In [ ]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect

In [ ]:
def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([[0, 0],[maxWidth - 1, 0],[maxWidth - 1, maxHeight - 1],[0, maxHeight - 1]], dtype = "float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

In [ ]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000

import warnings
warnings.simplefilter ('ignore', Image.DecompressionBombWarning)

from pdf2image import convert_from_path
dir_train = "../ResturantBill/train"
os.mkdir(os.path.join('../ResturantBill', "trainedImages"))

for pdfFile in os.listdir(dir_train):
    
    if(pdfFile.endswith(".pdf")):
        pages = convert_from_path(os.path.join(dir_train, pdfFile), 500)
        for page in pages:
            page.save(os.path.join('../ResturantBill/trainedImages', pdfFile.replace('.pdf', '.jpg')), 'JPEG')

In [1]:
import cv2
import numpy as np
import imutils
import pandas as pd 
  
import os

def crop_minAreaRect(img, rect):

    # rotate img
    angle = rect[2]
    rows,cols = img.shape[0], img.shape[1]
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
    img_rot = cv2.warpAffine(img,M,(cols,rows))

    # rotate bounding box
    rect0 = (rect[0], rect[1], 0.0) 
    box = cv2.boxPoints(rect0)
    pts = np.int0(cv2.transform(np.array([box]), M))[0]    
    pts[pts < 0] = 0

    # crop
    img_crop = img_rot[pts[1][1]:pts[0][1], 
                       pts[1][0]:pts[2][0]]

    return img_crop

def captch_ex(path, file_name):
    img = cv2.imread(path)
    img_area = img.shape[0] * img.shape[1]
    print(img_area)
    img_final = cv2.imread(file_name)
    img2gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(img2gray, 90, 180, cv2.THRESH_BINARY)
    image_final = cv2.bitwise_and(img2gray, img2gray, mask=mask)
    ret, new_img = cv2.threshold(image_final, 90, 180, cv2.THRESH_BINARY_INV)  # for black text , cv.THRESH_BINARY_INV
    kernel = np.ones((5,5), np.uint8)
    # apply some dilation and erosion to join the gaps - change iteration to detect more or less area's
    thresh = cv2.dilate(new_img, kernel, iterations = 70)
    thresh = cv2.erode(thresh, kernel, iterations = 2)
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    contours = imutils.grab_contours(contours)
    index = 0
    folder = "../Data-Cropped-Images/crop_"+file_name
    # Create the pandas DataFrame 
    df = pd.DataFrame(columns = ['Filename', 'X', 'Y', 'W', 'H', 'Angle'])
    pathToCSV = file_name+"_contours.csv"
    
    os.makedirs(folder)
    for contour in contours:
        
        min_rect = cv2.minAreaRect(contour)  # min_area_rectangle
        cropped_img = crop_minAreaRect(img, min_rect)
        file_name = 'cropped_'+ str(index) + '.jpg'
        pathToFile = os.path.join(os.path.abspath(folder), file_name)
        print(pathToFile)
#         print(min_rect[0][0], min_rect[0][1])
#         print(min_rect[1][0], min_rect[1][1])
        try:
            if min_rect[1][0] * min_rect[1][1] < img_area*0.9:
                cv2.imwrite(pathToFile, cropped_img)
                index = index + 1
#               write code here to add the coords of min_rect to a csv along with filename                
                df.loc[len(df)] = [file_name, min_rect[0][0], min_rect[0][1],min_rect[1][0], min_rect[1][1], min_rect[2]]
                
                min_rect = np.int0(cv2.boxPoints(min_rect))
                

        except Exception as e:
            continue
    df.to_csv(os.path.join(folder, pathToCSV), index=False)
    
dirTrainedImage = "../ResturantBill/trainedImages"
for imgFile in os.listdir(dirTrainedImage):
    if(imgFile.endswith(".jpg")):
        file_name = imgFile.split(".")[0]
        captch_ex(os.path.join(dirTrainedImage,file_name+".jpg"), file_name)

14819630
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_177/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_177/cropped_1.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_177/cropped_1.jpg
6052000
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_163/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_163/cropped_0.jpg
7315625
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_188/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_188/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_188/cropped_1.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_188/cropped_1.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_188/cropped_1.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_188/cropped_2.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_188/cropped_3.jpg
241

/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_171/cropped_24.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_171/cropped_25.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_171/cropped_25.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_171/cropped_26.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_171/cropped_26.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_171/cropped_26.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_171/cropped_26.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_171/cropped_27.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_171/cropped_28.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_171/cropped_29.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_171/cropped_30.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_171/cropped_31.jpg
/Users/megharana

/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_49/cropped_0.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_48/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_48/cropped_1.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_48/cropped_1.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_60/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_60/cropped_1.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_74/cropped_0.jpg
28056840
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_167/cropped_0.jpg
7337500
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_173/cropped_0.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_128/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_128/cropped_1.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_

/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_9/cropped_2.jpg
7337500
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_138/cropped_0.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_104/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_104/cropped_0.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_110/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_110/cropped_1.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_110/cropped_2.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_110/cropped_2.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_109/cropped_0.jpg
11961312
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_135/cropped_0.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_121/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/cro

24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_126/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_126/cropped_1.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_130/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_130/cropped_1.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_130/cropped_2.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_130/cropped_3.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_124/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_124/cropped_1.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_124/cropped_2.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_124/cropped_3.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_124/cropped_3.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_118/croppe

24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_91/cropped_0.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_90/cropped_0.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_84/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_84/cropped_1.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_84/cropped_2.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_84/cropped_3.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_84/cropped_4.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_53/cropped_0.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_47/cropped_0.jpg
2645676
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_183/cropped_0.jpg
31873871
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_154/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/c

/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_184/cropped_9.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_184/cropped_10.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_184/cropped_10.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_184/cropped_11.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_184/cropped_12.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_184/cropped_13.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_184/cropped_14.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_184/cropped_14.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_184/cropped_15.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_184/cropped_16.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_184/cropped_17.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_184/cropped_18.jpg
/Users/megharana/

24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_54/cropped_0.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_40/cropped_0.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_68/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_68/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_68/cropped_1.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_68/cropped_2.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_68/cropped_3.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_68/cropped_3.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_97/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_97/cropped_1.jpg
24163936
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_83/cropped_0.jpg
/Users/megharana/Documents/OCRDeep/Data-Cropped-Images/crop_TR_83/cropped_0

In [18]:
imgListWithNoContour = []

folderOfCropeedImages = "../Data-Cropped-Images"

for root, dirs, files in os.walk(folderOfCropeedImages):
    for image_dir in dirs:
        if len(os.listdir(os.path.join(folderOfCropeedImages, image_dir))) == 1:
            imgListWithNoContour.append(os.path.basename(image_dir))

    
    
    
    
    

print(imgListWithNoContour)

['crop_TR_91', 'crop_TR_53', 'crop_TR_30', 'crop_TR_37', 'crop_TR_90', 'crop_TR_64', 'crop_TR_137', 'crop_TR_106', 'crop_TR_164', 'crop_TR_100', 'crop_TR_138', 'crop_TR_131', 'crop_TR_165', 'crop_TR_149', 'crop_TR_182', 'crop_TR_178', 'crop_TR_148', 'crop_TR_112', 'crop_TR_46', 'crop_TR_3', 'crop_TR_4', 'crop_TR_83', 'crop_TR_70', 'crop_TR_23', 'crop_TR_15', 'crop_TR_12', 'crop_TR_85', 'crop_TR_82', 'crop_TR_76', 'crop_TR_49', 'crop_TR_40', 'crop_TR_5', 'crop_TR_47', 'crop_TR_61', 'crop_TR_59', 'crop_TR_66', 'crop_TR_92', 'crop_TR_50', 'crop_TR_32', 'crop_TR_93', 'crop_TR_169', 'crop_TR_160', 'crop_TR_167', 'crop_TR_158', 'crop_TR_134', 'crop_TR_102', 'crop_TR_161', 'crop_TR_168', 'crop_TR_135', 'crop_TR_186', 'crop_TR_172', 'crop_TR_144', 'crop_TR_118', 'crop_TR_129', 'crop_TR_142', 'crop_TR_180', 'crop_TR_174', 'crop_TR_173', 'crop_TR_20', 'crop_TR_27', 'crop_TR_29', 'crop_TR_16', 'crop_TR_80', 'crop_TR_28', 'crop_TR_17', 'crop_TR_10', 'crop_TR_21', 'crop_TR_75', 'crop_TR_72', 'crop_

In [40]:
import shutil

dir = 'path_to_my_folder'


def captch_ex_V2(path, file_name):
    img = cv2.imread(path)
    img_area = img.shape[0] * img.shape[1]
#     print(file_name)
#     print(path)
    img_final = cv2.imread(file_name)
    img2gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(img2gray, 90, 180, cv2.THRESH_BINARY)
    image_final = cv2.bitwise_and(img2gray, img2gray, mask=mask)
    ret, new_img = cv2.threshold(image_final, 90, 180, cv2.THRESH_BINARY_INV)  # for black text , cv.THRESH_BINARY_INV
    kernel = np.ones((5,5), np.uint8)
    # apply some dilation and erosion to join the gaps - change iteration to detect more or less area's
    thresh = cv2.dilate(new_img, kernel, iterations = 50)
    thresh = cv2.erode(thresh, kernel, iterations = 50)
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    contours = imutils.grab_contours(contours)
    index = 0
    folder = "../Data-Cropped-Images2/crop_"+file_name
    # Create the pandas DataFrame 
    df = pd.DataFrame(columns = ['Filename', 'X', 'Y', 'W', 'H', 'Angle'])
    pathToCSV = file_name+"_contours.csv"
    
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.makedirs(folder)
    for contour in contours:
        
        min_rect = cv2.minAreaRect(contour)  # min_area_rectangle
        cropped_img = crop_minAreaRect(img, min_rect)
        file_name = 'cropped_'+ str(index) + '.jpg'
        pathToFile = os.path.join(os.path.abspath(folder), file_name)
#         print(pathToFile)
#         print(min_rect[0][0], min_rect[0][1])
#         print(min_rect[1][0], min_rect[1][1])
        try:
            if min_rect[1][0] * min_rect[1][1] < img_area*0.9:
                cv2.imwrite(pathToFile, cropped_img)
                index = index + 1
#               write code here to add the coords of min_rect to a csv along with filename                
                df.loc[len(df)] = [file_name, min_rect[0][0], min_rect[0][1],min_rect[1][0], min_rect[1][1], min_rect[2]]
                
                min_rect = np.int0(cv2.boxPoints(min_rect))
                

        except Exception as e:
            continue
#     df.to_csv(os.path.join(folder, pathToCSV), index=False)
    
file_name = imgListWithNoContour[0].replace("crop_","")
captch_ex_V2(os.path.join(dirTrainedImage,file_name+".jpg"), file_name)    
    